In [81]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [82]:
class Preprocessor:
    def __init__(self):
        pass
    
    def __init__(self,df_identity, df_transactions):
        df = df_transactions.join(df_identity,how='left',on='TransactionID',lsuffix='_left', rsuffix='_right')
        df.drop(inplace=True,columns=['TransactionID_left','TransactionID_right'])
        self.X = df.iloc[:,1:]
        self.n = self.X.shape[1]
        self.y = df.iloc[:,0]
    
    def preprocess(self, weights, threshold) -> pd.DataFrame:
        pass


In [83]:
from sklearn.tree import DecisionTreeClassifier

class Estimator:
    def __init__(self):
        pass

    def get_score(X, y) -> float:
        pass

In [86]:
class GreyWolfOptimizer:
    
    def __init__(self,estimator: Estimator, preprocessor: Preprocessor):
        self.estimator = estimator
        self.preprocessor = preprocessor

    def _init_population(self, features_n, population_size):
        population = []
        individual = []
        for _ in range(population_size):
            individual = np.random.random(size=len(features_n))
            population.append(individual)
        return population

    def run(self, iterations = 50, threshold = 0.5, population_size = 10):
        
        #Init algo
        n = self.preprocessor.n # number of features
        population = self._init_population(population_size=population_size,features_n=n)

        bar = tqdm(range(iterations))

        for i in bar: #Epochs
            step = i * ((2-0) / iterations)
            population = sorted(population, key=lambda x: self._calculate_cost(x,threshold),reverse = True)
            X = population[:3] # alpha, beta, delta
            a = np.full((1, n), 2 - step)
            new_population = []
            for individual in population:
                r1 = np.random.random(size=n)
                r2 = np.random.random(size=n)
                A = 2 * a * r1 - a
                C = 2 * r2
                new_individual = np.full((1, n), 0.0)
                for x in X:
                    D = abs(C * x - individual)
                    new_x = x - A * D
                    new_individual = new_individual + new_x
                new_population.append(new_individual / 3)
            
            population = new_population

        # Best solution after n iterations
        population = sorted(population, key=lambda x: self._calculate_cost(x,threshold),reverse = True)
        X_alpha = population[0]
        return X_alpha


    def _calculate_cost(self, weights, threshold):
        X = self.preprocessor.preprocess(weights,threshold=threshold)
        y = self.preprocessor.y

        return self.estimator.get_score(X,y)
    